# SARCASM DETECTION EDA
this module contain twitter exploratory data analysis for training thew model.

## Import
Import required lib

In [3]:
# Import required lib
import re
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns
import string
import nltk
import warnings 
warnings.filterwarnings("ignore", category=DeprecationWarning)


## Load dataset 

In [4]:
train  = pd.read_csv('data/train_E6oV3lV.csv')
test = pd.read_csv('data/test_tweets_anuFYb8.csv')

In [5]:
train.head()

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation


## Preprocess dataset
IN the pre-processing part we will be cleaning dataset for further use. The pipeline is as follows:<br>
1. Reamoving @auther
2. Remove words with lenth less than 3
3. Removing Punctuations, Numbers, and Special Characters


In [6]:
def remove_pattern(input_txt, pattern):
    """
    Remove given pattern form text
    """
    r = re.findall(pattern, input_txt)
    for i in r:
        input_txt = re.sub(i, '', input_txt)
        
    return input_txt 

In [7]:
train['tweet'] = np.vectorize(remove_pattern)(train['tweet'], "@[\w]*")

In [8]:
train['tweet'] = train['tweet'].str.replace("[^a-zA-Z#]", " ")

In [9]:
train['tweet'] = train['tweet'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))

In [10]:
#preview after cleanming
train.head()


,id,label,tweet
0,1,0,when father dysfunctional selfish drags kids i...
1,2,0,thanks #lyft credit cause they offer wheelchai...
2,3,0,bihday your majesty
3,4,0,#model love take with time
4,5,0,factsguide society #motivation


## Data analysis
In this section data analysis is taken part

In [11]:
text=train['tweet'][3] #text for refernece
from nltk.tokenize import word_tokenize, sent_tokenize
sents=sent_tokenize(text)
print(sents)

['#model love take with time']


In [12]:
#tokenize sentance by words 
words=[word_tokenize(sent) for sent in sents]
print(words)

[['#', 'model', 'love', 'take', 'with', 'time']]


In [13]:
from nltk.corpus import stopwords 
from string import punctuation
customStopWords=set(stopwords.words('english')+list(punctuation))

In [14]:
#Removing stopwords
wordsWOStopwords=[word for word in word_tokenize(text) if word not in customStopWords]
print(wordsWOStopwords)

['model', 'love', 'take', 'time']


In [15]:
#taking bu-gram features
from nltk.collocations import *
bigram_measures = nltk.collocations.BigramAssocMeasures()
finder = BigramCollocationFinder.from_words(wordsWOStopwords)
sorted(finder.ngram_fd.items())

[(('love', 'take'), 1), (('model', 'love'), 1), (('take', 'time'), 1)]

In [16]:
#stemming words
from nltk.stem.lancaster import LancasterStemmer
st=LancasterStemmer()
stemmedWords=[st.stem(word) for word in word_tokenize(text)]
print(stemmedWords)

['#', 'model', 'lov', 'tak', 'with', 'tim']


In [17]:
# Word tagging
nltk.pos_tag(word_tokenize(text))

[('#', '#'),
 ('model', 'NN'),
 ('love', 'NNS'),
 ('take', 'VBP'),
 ('with', 'IN'),
 ('time', 'NN')]

In [18]:
# Extact meaning of words
from nltk.wsd import lesk
sense1 = lesk(word_tokenize(text),'model')
print(sense1, sense1.definition())

Synset('model.v.06') construct a model of


## Quick Theory validation
                

In [27]:
from collections import Counter
positive_counts = Counter()
negative_counts = Counter()
total_counts = Counter()

In [28]:
reviews = train['tweet']
labels = train['label']

In [47]:
for i in range(len(reviews)):
    if(labels[i] == 1):
        for word in reviews[i].split(" "):
            positive_counts[word] += 1
            total_counts[word] += 1
    else:
        for word in reviews[i].split(" "):
            negative_counts[word] += 1
            total_counts[word] += 1

In [48]:
# Examine the counts of the most common words in positive reviews
positive_counts.most_common(20)

[('this', 2875),
 ('with', 2635),
 ('that', 2266),
 ('your', 1779),
 ('have', 1745),
 ('happy', 1598),
 ('#love', 1552),
 ('just', 1440),
 ('will', 1346),
 ('when', 1309),
 ('what', 1269),
 ('love', 1236),
 ('like', 1169),
 ('from', 1131),
 ('time', 1124),
 ('today', 1033),
 ('about', 974),
 ('they', 937),
 ('people', 906),
 ('#positive', 878)]

In [49]:
# Examine the counts of the most common words in negative reviews
negative_counts.most_common(20)

[('this', 5162),
 ('with', 4922),
 ('that', 3920),
 ('your', 3294),
 ('have', 3175),
 ('happy', 3163),
 ('#love', 3071),
 ('just', 2643),
 ('will', 2470),
 ('when', 2459),
 ('love', 2424),
 ('what', 2235),
 ('time', 2182),
 ('from', 2031),
 ('today', 2024),
 ('like', 1921),
 ('#positive', 1756),
 ('about', 1630),
 ('they', 1562),
 ('good', 1561)]

In [50]:
pos_neg_ratios = Counter()

# Calculate the ratios of positive and negative uses of the most common words

for term,cnt in list(total_counts.most_common()):
    if(cnt > 100):
        pos_neg_ratio = positive_counts[term] / float(negative_counts[term]+1)
        pos_neg_ratios[term] = pos_neg_ratio

In [61]:
print("Pos-to-neg ratio for 'gorgeous' = {}".format(pos_neg_ratios["gorgeous"]))
print("Pos-to-neg ratio for 'girls' = {}".format(pos_neg_ratios["girls"]))
print("Pos-to-neg ratio for 'racism' = {}".format(pos_neg_ratios["racism"]))

Pos-to-neg ratio for 'gorgeous' = 0.4927536231884058
Pos-to-neg ratio for 'girls' = 0.6123188405797102
Pos-to-neg ratio for 'racism' = 1.632183908045977


The ratios tell us which words are used more often in Sarcastic reviews, but the specific values we've calculated are a bit difficult to work with.<br>
`To fix these issues, we'll convert all of our ratios to new values using logarithms.`

In [62]:
for word,ratio in pos_neg_ratios.most_common():
    pos_neg_ratios[word] = np.log(ratio)

In [63]:
print("Pos-to-neg ratio for 'gorgeous' = {}".format(pos_neg_ratios["gorgeous"]))
print("Pos-to-neg ratio for 'girls' = {}".format(pos_neg_ratios["girls"]))
print("Pos-to-neg ratio for 'racism' = {}".format(pos_neg_ratios["racism"]))

Pos-to-neg ratio for 'gorgeous' = -0.7077459799810979
Pos-to-neg ratio for 'girls' = -0.49050215079407644
Pos-to-neg ratio for 'racism' = 0.48991893894667704


In [66]:
pos_neg_ratios.most_common(30)

[('#allahsoil', 0.6823362644557297),
 ('#sjw', 0.6797241602278047),
 ('#libtard', 0.660990068925414),
 ('libtard', 0.6405034470745233),
 ('#liberal', 0.6237186741281824),
 ('stomping', 0.6146755651184501),
 ('#politics', 0.6095604644516797),
 ('#tampa', 0.5787368293822011),
 ('racism', 0.48991893894667704),
 ('racist', 0.3214462520549885),
 ('#trump', 0.3011887712911459),
 ('#maga', 0.29334780998745824),
 ('#miami', 0.2525907526405108),
 ('#retweet', 0.24946085963158324),
 ('white', 0.23853470163450707),
 ('#obama', 0.1823215567939546),
 ('#black', 0.13846967426510512),
 ('comments', 0.09333193979221913),
 ('#hate', 0.08288765980576765),
 ('#brexit', 0.06899287148695142),
 ('anti', 0.018349138668196617),
 ('against', 0.005277057100843819),
 ('obama', -0.00722024797348702),
 ('listen', -0.011560822401075971),
 ('black', -0.0418179396833216),
 ('women', -0.04652001563489282),
 ('suppoers', -0.08594242980072477),
 ('woman', -0.1038995950181515),
 ('#leadership', -0.11934675763256625),
 ('

In [65]:
list(reversed(pos_neg_ratios.most_common()))[0:30]


[('#workout', -0.7077459799810979),
 ('#fit', -0.7077459799810979),
 ('#chill', -0.7077459799810979),
 ('website', -0.7077459799810979),
 ('ness', -0.7077459799810979),
 ('#positivity', -0.7077459799810979),
 ('gorgeous', -0.7077459799810979),
 ('#essentialoils', -0.7077459799810979),
 ('planning', -0.7077459799810979),
 ('#sunset', -0.7077459799810979),
 ('yours', -0.7077459799810979),
 ('launch', -0.7077459799810979),
 ('dory', -0.7077459799810979),
 ('memories', -0.7077459799810979),
 ('#igers', -0.7077459799810979),
 ('#snapchat', -0.7077459799810979),
 ('florida', -0.7077459799810979),
 ('begins', -0.7077459799810979),
 ('feelings', -0.7077459799810979),
 ('christina', -0.7077459799810979),
 ('#run', -0.7077459799810979),
 ('august', -0.7073318155519017),
 ('#pray', -0.7073318155519017),
 ('broke', -0.7073318155519017),
 ('#heabroken', -0.7073318155519017),
 ('#behappy', -0.7073318155519017),
 ('#blogger', -0.7073318155519017),
 ('#youtube', -0.7073318155519017),
 ('#animals', -0.

common words like "the" appear very often in both positive and negative reviews. Instead of finding the most common words in positive or negative reviews, what you really want are the words found in positive reviews more often than in negative reviews, and vice versa. To accomplish this, you'll need to calculate the **ratios** of word usage between positive and negative reviews.